(howto-measurement-control)=
# Measurement Control

We first prepare some utilities necessarily for the examples.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from qcodes import ManualParameter, Parameter

import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl

dh.set_datadir(Path.home() / "quantify-data")
meas_ctrl = MeasurementControl("meas_ctrl")

par0 = ManualParameter(name="x0", label="X0", unit="s")
par1 = ManualParameter(name="x1", label="X1", unit="s")
par2 = ManualParameter(name="x2", label="X2", unit="s")
par3 = ManualParameter(name="x3", label="X3", unit="s")
sig = Parameter(name="sig", label="Signal", unit="V", get_cmd=lambda: np.exp(par0()))

## Comparing iterative and batched execution loop

### Iterative settables only

In [ ]:
par0.batched = False
par1.batched = False
par2.batched = False

sig.batched = False

meas_ctrl.settables([par0, par1, par2])
meas_ctrl.setpoints_grid(
    [
        np.linspace(0, 1, 4),
        np.linspace(1, 2, 5),
        np.linspace(2, 3, 6),
    ]
)
meas_ctrl.gettables(sig)
dset = meas_ctrl.run("demo")
list(xr.plot.line(xi, label=name) for name, xi in dset.coords.items())
plt.gca().legend()

### Batched settables only

Note that the settable with lowest `.batch_size`  will be correspond to the innermost loop.

In [ ]:
par0.batched = True
par1.batch_size = 8
par1.batched = True
par1.batch_size = 8
par2.batched = True
par2.batch_size = 4

sig = Parameter(name="sig", label="Signal", unit="V", get_cmd=lambda: np.exp(par2()))
sig.batched = True
sig.batch_size = 32

meas_ctrl.settables([par0, par1, par2])
meas_ctrl.setpoints_grid(
    [
        np.linspace(0, 1, 3),
        np.linspace(1, 2, 5),
        np.linspace(2, 3, 4),
    ]
)
meas_ctrl.gettables(sig)
dset = meas_ctrl.run("demo")
list(xr.plot.line(xi, label=name) for name, xi in dset.coords.items())
plt.gca().legend()

### Mixed batched and iterative settables

Note that the settable with lowest `.batch_size`  will be correspond to the innermost loop.
Furthermore, the iterative settables will be the outermost loops.

In [ ]:
par0.batched = False
par1.batched = True
par1.batch_size = 8
par2.batched = False
par3.batched = True
par3.batch_size = 4

sig = Parameter(name="sig", label="Signal", unit="V", get_cmd=lambda: np.exp(par3()))
sig.batched = True
sig.batch_size = 32

meas_ctrl.settables([par0, par1, par2, par3])
meas_ctrl.setpoints_grid(
    [
        np.linspace(0, 1, 3),
        np.linspace(1, 2, 5),
        np.linspace(2, 3, 4),
        np.linspace(3, 4, 6),
    ]
)
meas_ctrl.gettables(sig)
dset = meas_ctrl.run("demo")
list(xr.plot.line(xi, label=name) for name, xi in dset.coords.items())
plt.gca().legend()

(howto-measurement-control-insmon)=
## Instrument Monitor

You can instantiate an instrument monitor in the following way:

In [ ]:
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization import InstrumentMonitor

instrument_monitor = InstrumentMonitor("instrument_monitor")
# Set True if you want to query the instruments about each parameter
# before updating the window. Can be slow due to communication overhead.
instrument_monitor.update_snapshot(False)